In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot  as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as Data

torch.manual_seed(42)

In [10]:
####### Model Creation #######

# Get cpu or gpu device for training (If capable GPU present).
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device} device available!")

class NeuralNetwork(nn.Module):

    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.hidden1 = torch.nn.Linear(4, 50) # hidden layer
        self.hidden2 = torch.nn.Linear(50, 4) # hidden layer
        self.out = torch.nn.Linear(4, 1)     # output layer

    def forward(self, x):
        z = F.relu(self.hidden1(x)) # activation function for first hidden layer
        z = F.relu(self.hidden2(z)) # activation function for second hidden layer
        z = self.out(z)        # linear output
        return z

model = NeuralNetwork().to('cpu')
print(model)

cuda device available!
NeuralNetwork(
  (hidden1): Linear(in_features=4, out_features=50, bias=True)
  (hidden2): Linear(in_features=50, out_features=4, bias=True)
  (out): Linear(in_features=4, out_features=1, bias=True)
)


In [3]:
####### Dataset Loading #######

dataset = pd.read_csv("Data/BTC-USD.csv")
print(dataset.shape)
dataset.head()

(1827, 7)


,Date,Open,High,Low,Close,Adj Close,Volume
0,2016-12-21,800.643982,834.281006,799.405029,834.281006,834.281006,155576000
1,2016-12-22,834.179993,875.781982,834.148987,864.539978,864.539978,200027008
2,2016-12-23,864.888000,925.117004,864.677002,921.984009,921.984009,275564000
3,2016-12-24,922.179993,923.479004,886.335022,898.822021,898.822021,137727008
4,2016-12-25,899.651978,899.651978,862.424011,896.182983,896.182983,143664992


In [4]:
####### Scaling #######

scaled_data = dataset[['Open', 'High', 'Low', 'Close', 'Volume']]
scaler = MinMaxScaler(copy=False)
scaled_data[['Open', 'High', 'Low', 'Close', 'Volume']] = scaler.fit_transform(scaled_data[['Open', 'High', 'Low', 'Close', 'Volume']])
scaled_data

D:\Users\kkats\anaconda3\envs\ml\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


,Open,High,Low,Close,Volume
0,0.000381,0.000161,0.000665,0.000846,0.000270
1,0.000884,0.000772,0.001195,0.001299,0.000397
2,0.001343,0.001498,0.001660,0.002159,0.000612
3,0.002201,0.001474,0.001990,0.001813,0.000219
4,0.001864,0.001123,0.001625,0.001773,0.000236
...,...,...,...,...,...
1822,0.702042,0.694191,0.683605,0.680117,0.093591
1823,0.680560,0.684023,0.683304,0.689799,0.074200
1824,0.690064,0.695438,0.697086,0.687676,0.071510
1825,0.687865,0.685316,0.683020,0.690270,0.088060


In [5]:
X = torch.Tensor(scaled_data[['Open', 'High', 'Low', 'Volume']].values)
y = torch.Tensor(scaled_data['Close'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

torch.Size([1461, 4])
torch.Size([1461])
torch.Size([366, 4])
torch.Size([366])


In [9]:
####### Data Preperation #######

BATCH_SIZE = 50

train_dataset = Data.TensorDataset(X_train, y_train)
test_dataset = Data.TensorDataset(X_test, y_test)

train_loader = Data.DataLoader(
    dataset=train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False)

test_loader = Data.DataLoader(
    dataset=test_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_func = torch.nn.MSELoss() 

In [ ]:
####### Training #######

EPOCH = 200
running_loss = 0.0

for epoch in range(EPOCH):
    for step, (batch_x, batch_y) in enumerate(train_loader): # for each training step
        
        optimizer.zero_grad()   # clear gradients for next train

        predictions = model(batch_x)           # input x and predict based on x
        loss = loss_func(predictions, batch_y)  # calculate MSE based on x and y
        loss.backward()                        # backpropagation, compute gradients
        optimizer.step()                       # apply gradients to weights

        # print statistics
        running_loss += loss.item()
        if step % 20 == 19:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, step + 1, running_loss / 2000))
            running_loss = 0.0

In [ ]:
####### Testing #######

